**ACTIVIDAD N° 06 - PROBLEMA EXTENDIDO**

In [3]:
import random
import numpy as np
import pandas as pd

# ====================== LECTURA DE DATOS ======================
df = pd.read_csv('../notas_1u.csv')
alumnos = df['Alumno'].tolist()
notas = df['Nota'].tolist()

# ====================== CONFIGURACIÓN GENÉTICA ======================
NUM_ALUMNOS = 39
NUM_EXAMENES = 4  # Cambio importante: ahora tenemos 4 exámenes
GENES_POR_ALUMNO = NUM_EXAMENES  # 4 bits por alumno
LONGITUD_CROMOSOMA = NUM_ALUMNOS * GENES_POR_ALUMNO

# ====================== FUNCIONES ======================
def crear_cromosoma():
    cromosoma = []
    for _ in range(NUM_ALUMNOS):
        examen = random.randint(0, NUM_EXAMENES - 1)
        genes = [0] * NUM_EXAMENES
        genes[examen] = 1
        cromosoma.extend(genes)
    return cromosoma

def decodificar_cromosoma(cromosoma):
    asignaciones = {ex: [] for ex in ['A', 'B', 'C', 'D']}
    examenes = list(asignaciones.keys())
    for i in range(NUM_ALUMNOS):
        idx = i * GENES_POR_ALUMNO
        for j in range(NUM_EXAMENES):
            if cromosoma[idx + j] == 1:
                asignaciones[examenes[j]].append(i)
                break
    return asignaciones

def calcular_fitness(cromosoma):
    asignaciones = decodificar_cromosoma(cromosoma)
    tamaños = sorted([len(asignaciones[ex]) for ex in asignaciones])
    
    # Distribución más equitativa con 39 alumnos en 4 grupos: [9, 10, 10, 10]
    if tamaños != [9, 10, 10, 10]:
        return -1000

    promedios = []
    for examen in asignaciones:
        notas_examen = [notas[i] for i in asignaciones[examen]]
        promedios.append(np.mean(notas_examen))

    desviacion = np.std(promedios)
    return -desviacion  # Menor desviación es mejor

def mutacion(cromosoma):
    crom_mutado = cromosoma.copy()
    alumno1, alumno2 = random.sample(range(NUM_ALUMNOS), 2)
    idx1, idx2 = alumno1 * GENES_POR_ALUMNO, alumno2 * GENES_POR_ALUMNO

    ex1 = cromosoma[idx1:idx1 + GENES_POR_ALUMNO].index(1)
    ex2 = cromosoma[idx2:idx2 + GENES_POR_ALUMNO].index(1)

    if ex1 != ex2:
        crom_mutado[idx1:idx1 + GENES_POR_ALUMNO] = [0] * GENES_POR_ALUMNO
        crom_mutado[idx1 + ex2] = 1
        crom_mutado[idx2:idx2 + GENES_POR_ALUMNO] = [0] * GENES_POR_ALUMNO
        crom_mutado[idx2 + ex1] = 1

    return crom_mutado

def algoritmo_genetico(generaciones=100, tam_poblacion=50):
    poblacion = [crear_cromosoma() for _ in range(tam_poblacion)]

    for gen in range(generaciones):
        fitness_scores = [(crom, calcular_fitness(crom)) for crom in poblacion]
        fitness_scores.sort(key=lambda x: x[1], reverse=True)

        nueva_poblacion = [fs[0] for fs in fitness_scores[:int(tam_poblacion * 0.2)]]

        while len(nueva_poblacion) < tam_poblacion:
            padre = random.choice(nueva_poblacion)
            hijo = mutacion(padre)
            nueva_poblacion.append(hijo)

        poblacion = nueva_poblacion

        if gen % 20 == 0:
            print(f"Generación {gen}: Mejor fitness = {fitness_scores[0][1]:.4f}")

    return fitness_scores[0][0]

# ====================== EJECUCIÓN PRINCIPAL ======================
print("REPRESENTACIÓN BINARIA - 4 EXÁMENES")
print("Problema: Distribuir 39 alumnos en 4 exámenes (A, B, C, D)")
print("Cromosoma: 156 bits (39 alumnos × 4 bits cada uno)")
print("Gen: [0,0,1,0] significa alumno asignado al examen C\n")

mejor_solucion = algoritmo_genetico()
asignaciones_finales = decodificar_cromosoma(mejor_solucion)

print("\nDistribución final:")
promedios = []
for examen in ['A', 'B', 'C', 'D']:
    indices = asignaciones_finales[examen]
    notas_examen = [notas[i] for i in indices]
    promedio = np.mean(notas_examen)
    promedios.append(promedio)
    print(f"Examen {examen}: {len(indices)} alumnos, promedio = {promedio:.2f}")
    print(f"  Ejemplos: {[alumnos[i] for i in indices[:5]]}...")

print(f"\nDesviación estándar entre promedios: {np.std(promedios):.4f}")



REPRESENTACIÓN BINARIA - 4 EXÁMENES
Problema: Distribuir 39 alumnos en 4 exámenes (A, B, C, D)
Cromosoma: 156 bits (39 alumnos × 4 bits cada uno)
Gen: [0,0,1,0] significa alumno asignado al examen C

Generación 0: Mejor fitness = -0.2693
Generación 20: Mejor fitness = -0.0192
Generación 40: Mejor fitness = -0.0192
Generación 60: Mejor fitness = -0.0192
Generación 80: Mejor fitness = -0.0192

Distribución final:
Examen A: 10 alumnos, promedio = 15.40
  Ejemplos: ['Alumno6', 'Alumno7', 'Alumno9', 'Alumno17', 'Alumno18']...
Examen B: 10 alumnos, promedio = 15.40
  Ejemplos: ['Alumno3', 'Alumno4', 'Alumno5', 'Alumno8', 'Alumno12']...
Examen C: 9 alumnos, promedio = 15.44
  Ejemplos: ['Alumno1', 'Alumno11', 'Alumno13', 'Alumno15', 'Alumno26']...
Examen D: 10 alumnos, promedio = 15.40
  Ejemplos: ['Alumno2', 'Alumno10', 'Alumno16', 'Alumno24', 'Alumno25']...

Desviación estándar entre promedios: 0.0192


***ANALISIS COMPARATIVO***

## Comparación entre la versión de 3 exámenes y 4 exámenes

### Distribución de alumnos
- **3 Exámenes**: 13 alumnos por grupo → distribución perfectamente equilibrada.
- **4 Exámenes**: 10, 10, 10, 9 → distribución casi equitativa (un examen con un alumno menos).

### Fitness máximo alcanzado (menor desviación estándar)
- **3 Exámenes**: puede llegar a **0.0000** si los promedios de notas por grupo son idénticos.
- **4 Exámenes**: fitness más bajo alcanzado fue **-0.0192** → excelente, aunque no perfectamente equilibrado por la limitación natural del número impar (39 alumnos).

### Complejidad computacional
- **3 Exámenes**: cromosoma de **117 bits** (39 alumnos × 3 bits). Menor espacio de búsqueda.
- **4 Exámenes**: cromosoma de **156 bits** (39 alumnos × 4 bits). Mayor espacio de búsqueda → requiere más exploración evolutiva.

### Conclusión
- La versión con **3 exámenes** es más eficiente y precisa para 39 alumnos.
- Sin embargo, la versión con **4 exámenes** logró una distribución muy cercana al equilibrio (**desviación estándar = 0.0192**), lo que demuestra que el algoritmo es robusto incluso en condiciones menos favorables.
- La elección depende del contexto: usar **4 exámenes** es aceptable si el número total de alumnos se aproxima a un múltiplo de 4.


### CUESTIONARIO ADICIONAL

---

#### ¿Qué cambios necesitas hacer en el cromosoma?

Para extender el problema de asignación de 3 a 4 exámenes (A, B, C, D), se realizaron los siguientes cambios estructurales en el algoritmo genético:

- **Estructura del cromosoma**:  
  - **Antes**: 3 bits por alumno → cromosoma total de 117 bits (39 alumnos × 3).  
  - **Ahora**: 4 bits por alumno → cromosoma total de 156 bits (39 alumnos × 4).

- **Codificación**:  
  Cada alumno es representado con un gen de 4 bits. Ejemplo:  
  `[0, 0, 1, 0]` indica que el alumno fue asignado al examen C.

- **Modificaciones adicionales**:
  - `crear_cromosoma()` genera 4 bits por alumno.
  - `decodificar_cromosoma()` interpreta bloques de 4 bits.
  - `mutacion()` intercambia asignaciones entre exámenes A–D.
  - `calcular_fitness()` ahora valida que haya una distribución de 10–10–10–9, la más equitativa posible para 39 alumnos.

---

#### ¿Cómo afecta esto a la convergencia del algoritmo?

- **Mayor complejidad**:
  - Aumenta el tamaño del espacio de búsqueda.
  - La representación de 4 exámenes genera más combinaciones posibles.

- **Restricción natural**:
  - Con 39 alumnos y 4 grupos, **no es posible una división perfectamente equitativa** (porque 39 no es múltiplo de 4).  
  - Esto impide alcanzar un fitness ideal de `0.0000`.

- **Resultado observado**:
  - El mejor fitness obtenido fue `-0.0192`, lo cual implica una **desviación estándar entre promedios de notas muy baja**.
  - El algoritmo converge hacia una solución buena pero no perfecta.

---

#### Conclusión

- El cambio en la estructura del cromosoma fue necesario para representar un cuarto grupo.
- Esto **incrementó la dificultad y el tiempo de convergencia**, pero el algoritmo fue capaz de encontrar una **solución eficiente y casi equitativa**.
- Aun así, para 39 alumnos, **la versión de 3 exámenes sigue siendo más adecuada** por:
  - Mayor equilibrio posible.
  - Menor complejidad computacional.
  - Mejor capacidad de alcanzar un fitness óptimo.
